In [13]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import special
from toolbox import sphereical_transport, error
#inputs
n = 8
cells = 100
r = 1.0
scat_order = 0
sig_t = 3.0
sig_a = 1.0
q_0 = np.ones(cells) #p/cm^3-s
normalize = False
psi_incident = np.ones(int(n/2))/sig_a/2
accelerate = True
legendre_xs_mom = np.array([2])
#generate angular grid center points (roots of legendre polynomial of degree n) and the
#corresponding gauss quadrature weights
ang_cell_centers, w = np.polynomial.legendre.leggauss(n)
#generate angular cell edge values 
ang_cell_edges = np.zeros(n+1)
ang_cell_edges[0] = -1
for i in range(1,n+1):
    ang_cell_edges[i] = ang_cell_edges[i-1] + w[i-1]
#generate alpha coefficients
alpha = np.zeros(n+1)
for i in range(1,n+1):
    alpha[i] = alpha[i-1] - (2*w[i-1]*ang_cell_centers[i-1])
alpha[n] = 0
#generate beta coefficients
beta = np.zeros(n)
for i in range(n):
    beta[i] = (ang_cell_centers[i] - ang_cell_edges[i])/(ang_cell_edges[i+1] - ang_cell_edges[i])
#initialize sphereical transport class
sph_trans = sphereical_transport(q_0,legendre_xs_mom,cells,n,r,sig_t,sig_a,normalize,psi_incident,scat_order,ang_cell_centers,w,alpha,beta,accelerate)
#initial guess for source
ang_flux_0 = np.zeros(n)
for i in range(cells):
    sph_trans.update_scat_source_mom(i,ang_flux_0)
#generate initial phi using initial guess for psi
phi_0 = np.zeros(cells)
for i in range(cells):
    for ang in range(np.size(ang_flux_0)):
        phi_0[i] += ang_flux_0[ang]*w[ang]
epsilon = 1
#begin source iteration
index = 0
while epsilon > 1e-4:
    #compute psi at mu = -1 which will be the first angular inflow vector
    ang_inflow, origin_flux = sph_trans.starting_direction()
    #do the full domain sweep
    phi = sph_trans.sweep(ang_inflow,origin_flux)
    if accelerate:
        sph_trans.DSA(phi,phi_0)
    index += 1
    if index > 99:
        print('Iteration limit exceeded')
        break
    epsilon = np.max(error.rel_change(phi,phi_0))
    phi_0 = phi
print(phi)
print(index-1)
#compute balance tables
abs_rate = 0
for i in range(cells):
    abs_rate += phi[i]*sig_a*sph_trans.V(i)
reflection = 0
for ang in range(int(n/2),n):
    reflection += sph_trans.outflow[ang - int(n/2)]*ang_cell_centers[ang]*w[ang]*sph_trans.A(cells)
sink_rate = abs_rate + reflection
source_rate = 0
for i in range(cells):
    source_rate += q_0[i]*sph_trans.V(i)
for ang in range(0,int(n/2)):
    source_rate += sph_trans.psi_incident[ang]*-ang_cell_centers[ang]*w[ang]*sph_trans.A(cells)
bal = np.abs((source_rate - sink_rate)/source_rate)
print(source_rate)
print(abs_rate)
print(reflection)
print(bal)

[0.99999872 0.99999873 0.99999873 0.99999875 0.99999876 0.99999879
 0.99999881 0.99999885 0.99999889 0.99999895 0.999999   0.99999907
 0.99999915 0.99999923 0.99999932 0.99999941 0.99999951 0.99999962
 0.99999973 0.99999985 0.99999997 1.0000001  1.00000023 1.00000036
 1.0000005  1.00000063 1.00000077 1.00000091 1.00000105 1.00000119
 1.00000133 1.00000146 1.0000016  1.00000173 1.00000186 1.00000198
 1.0000021  1.00000222 1.00000233 1.00000243 1.00000253 1.00000262
 1.0000027  1.00000277 1.00000283 1.00000289 1.00000293 1.00000296
 1.00000298 1.00000298 1.00000298 1.00000296 1.00000292 1.00000287
 1.00000281 1.00000273 1.00000264 1.00000253 1.0000024  1.00000226
 1.00000211 1.00000194 1.00000175 1.00000155 1.00000134 1.00000112
 1.00000088 1.00000063 1.00000038 1.00000012 0.99999985 0.99999958
 0.99999932 0.99999905 0.99999879 0.99999854 0.99999831 0.99999808
 0.99999788 0.9999977  0.99999755 0.99999744 0.99999735 0.99999731
 0.99999731 0.99999736 0.99999746 0.9999976  0.9999978  0.9999

In [2]:
outflows = [4.1299017922967245,4.129801501299079,4.129776425395114]
order = np.abs((outflows[1]-outflows[0])/(outflows[1]-outflows[2]))
print(order)

3.999496799236155
